In [ ]:
# Quick setup
import sys
import pandas as pd
import numpy as np
from src.utils.notebook_setup import setup_notebook_environment
dbs, logger = await setup_notebook_environment()

# Now ready to work
logger.info("Starting data ingestion...")


In [ ]:
# Load CSV -> Insert into Bronze layer

df = pd.read_csv("../data/raw/heart_disease_uci.csv")

logger.debug(df.head())
logger.debug(df.shape)
logger.debug(df.columns)
logger.debug(df.dtypes)
logger.debug(df.isnull().sum())

# Insert into Bronze layer
data_dict = df.to_dict("records")
success = await dbs.insert_bronze_data(data_dict)

bronze_data = await dbs.get_bronze_data()
bronze_data_df = pd.DataFrame(bronze_data)
logger.info(bronze_data_df.head())

logger.info("=" * 60)
logger.info("BRONZE LAYER DATA SUMMARY")
logger.info("=" * 60)
logger.info(f"Final dataset shape: {bronze_data_df.shape}")
# Show final columns
logger.info(f"Final columns: {list(bronze_data_df.columns)}")

# Save to Bronze collection in MongoDB
logger.info("Saving raw unprocessed data to Bronze layer...")
